In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import lanelet2
from lanelet2.core import (AllWayStop, AttributeMap, BasicPoint2d,
                            BoundingBox2d, Lanelet, LaneletMap,
                            LaneletWithStopLine, LineString2d, LineString3d, Point2d, Point3d,
                            RightOfWay, TrafficLight, getId)
from lanelet2.projection import (UtmProjector, MercatorProjector,
                                    LocalCartesianProjector, GeocentricProjector)

import numpy as np

In [18]:
a = {}
a[1] = {'1':10, '2':13}
a[2] = {'1':2, '2':3}
a[3] = {'1':-2, '2':3}

print(a.keys())
print(a.values())
print(a.items())
sorted(a.items(), key=lambda item: item[1]['1'])

dict_keys([1, 2, 3])
dict_values([{'1': 10, '2': 13}, {'1': 2, '2': 3}, {'1': -2, '2': 3}])
dict_items([(1, {'1': 10, '2': 13}), (2, {'1': 2, '2': 3}), (3, {'1': -2, '2': 3})])


[(3, {'1': -2, '2': 3}), (2, {'1': 2, '2': 3}), (1, {'1': 10, '2': 13})]

In [3]:
def gen_point(pt):
    '''
    Generate a Lanelet2::Point2d object from a tuple/list/nparray of 2d coordinates
    '''
    
    return Point3d(getId(), float(pt[0]), float(pt[1]), 0.0)

def gen_line_string(pts):
    '''
    Generate a Lanelet2::LineString2d object from a list of array of 2d coordinates
    pts: [2,n] array
    '''
    d,n = pts.shape
    assert d == 2, "pts should be a [n,2] array"
    line_string = LineString3d(getId(),[])
    line_string.attributes["type"] = "line_thin"
    line_string.attributes["subtype"] = "dashed"
    for i in range(n):
        line_string.append(gen_point(pts[:,i]))
    return line_string

def gen_lanlet(left_pts, right_pts):
    left_line_string = gen_line_string(left_pts)
    right_line_string = gen_line_string(right_pts)
    lanelet = Lanelet(getId(), left_line_string, right_line_string)
    # lanelet.attributes["vehicle"] = "yes"
    lanelet.attributes["type"] = "lanelet"
    lanelet.attributes["subtype"] = "road"
    lanelet.attributes["location"] = "urban"
    return lanelet
    

In [163]:
r1 = np.array([[0,5],
                [0,0]])
c1 = np.array([[0,5],
                [3,3]])
l1 = np.array([[0,5],
                [6,6]])

r2 = np.array([[5,10],
                [0,1]])
l2 = np.array([[5,10],
                [3,4]])

r3 = np.array([[5,10],
                [0,-4]])
l3 = np.array([[5,10],
                [3,-1]])


r4 = np.array([[10,15],
                [-4,-4]])
l4 = np.array([[10,15],
                [-1,-1]])

r5 = np.array([[10,15],
                [4,4]])
l5 = np.array([[10,15],
                [1,1]])


In [4]:
pt1_l = gen_point([0,0])
pt2_l = gen_point([5,0])
pt3_l = gen_point([10,0])

pt1_r = gen_point([0,3])
pt2_r = gen_point([5,3])
pt3_r = gen_point([10,3])

l1_l = LineString3d(getId(),[pt1_l,pt2_l])
l2_l = LineString3d(getId(),[pt2_l,pt3_l])

l1_r = LineString3d(getId(),[pt1_r,pt2_r])
l2_r = LineString3d(getId(),[pt2_r,pt3_r])

lane1 = Lanelet(getId(),l1_l,l1_r)
lane2 = Lanelet(getId(),l2_l,l2_r)


In [7]:
projector = LocalCartesianProjector(lanelet2.io.Origin(0, 0, 0))


map = LaneletMap()
path = "test.osm"
# lane1 = gen_lanlet(l1, c1)
# lane1_left = gen_lanlet(c1, l1)
# lane2 = gen_lanlet(l2, r2)
# lane3 = gen_lanlet(l3, r3)
# lane4 = gen_lanlet(l4, r4)
# lane5 = gen_lanlet(l5, r5)

map.add(lane1)
# map.add(lane1_left)
map.add(lane2)
# map.add(lane3)
# map.add(lane4)
# map.add(lane5)


traffic_rules = lanelet2.traffic_rules.create(lanelet2.traffic_rules.Locations.Germany,
                                                lanelet2.traffic_rules.Participants.Vehicle)

graph = lanelet2.routing.RoutingGraph(map, traffic_rules) 
graph.checkValidity()

lanelet2.io.writeRobust(path, map, projector)


[]

In [121]:
loadmap = lanelet2.io.load(path, projector)
for lanelet in loadmap.laneletLayer:
    print(lanelet.leftBound[0].x, lanelet.leftBound[0].y)


10.000000176788873 4.000000101510982
10.000000176788873 -0.9999994725062403
4.999999531793911 2.9999995232588974
4.999999531793911 2.9999995232588974
0.0 2.9999995232579693
